# Project 3 - Satellite imagery with rasterio _et al._

![GDAL logo](img/gdal-logo.png)

**rasterio** is a Python library to read and write geospatial gridded raster datasets such as satellite imagery and terrain models. It wraps the raster capabilities of GDAL, a very mature and well-established software made by the  Open Source Geospatial Foundation. It is often used in combination with **fiona** (for vector data, wrapping GDAL/OGR), **Shapely** (for planar geometries, wrapping GEOS) and **pyproj** (for cartographic projections and coordinate transformations, wrapping PROJ).

- **Documentation**:
    - https://rasterio.readthedocs.io/
    - https://fiona.readthedocs.io/
    - https://shapely.readthedocs.io/
    - https://pyproj4.github.io/pyproj/
- **Source code**:
    - https://github.com/mapbox/rasterio/
    - https://github.com/Toblerity/Fiona/
    - https://github.com/Toblerity/Shapely
    - https://github.com/pyproj4/pyproj

## Obtaining raster images

We will make use of **Sentinel 2** images over the city of Madrid and its surroundings.

> Sentinel-2 is an Earth observation mission from the Copernicus Programme that systematically acquires optical imagery at high spatial resolution (10 m to 60 m) over land and coastal waters. The mission is a constellation with two twin satellites, Sentinel-2A and Sentinel-2B.

> One of the benefits of the Copernicus Programme is that the data and information produced in the framework of Copernicus are made available free-of-charge to all its users and the public, thus allowing downstream services to be developed.

Sentinel data can be obtained from Python using <a href="https://pypi.org/project/sentinelsat/"><code>sentinelsat</code></a>:

In [ ]:
import os
from IPython.display import GeoJSON

from sentinelsat import read_geojson

And we will look for images covering downtown Madrid:

In [ ]:
GeoJSON(read_geojson("search_polygon.geojson"))

In [ ]:
# If you have a sentinel account, you can use this script
# %run download_data.py
# Alternatively, you can download it using the cell below

In [ ]:
# This takes approximately 2 minutes, depending on Internet connection
import urllib.request

product_title = "S2B_MSIL2A_20210930T105749_N0301_R094_T30TVK_20210930T125620"
product_filename = "S2B_MSIL2A_20210930T105749_N0301_R094_T30TVK_20210930T125620.SAFE"

urllib.request.urlretrieve(
    "https://s2-storage.fra1.digitaloceanspaces.com/S2B_MSIL2A_20210930T105749_N0301_R094_T30TVK_20210930T125620_MANUAL.zip",
    "data/S2B_MSIL2A_20210930T105749_N0301_R094_T30TVK_20210930T125620.zip"
)

And finally, we unzip the data:

In [ ]:
from pathlib import Path
import shutil

product_file_path = Path("data") / (product_title + ".zip")

shutil.unpack_archive(product_file_path, "./data")

## Reading raster data

The True Color Image (TCI) corresponding to our data looks more or less like this:

![Sentinel Madrid](img/sentinel-madrid.png)

However, **we will not load it entirely in memory**, because the data is much bigger than our available RAM. Instead, we will use a window to load the data partially.

In [ ]:
import rasterio

true_color_path = list((Path("data") / product_filename).glob("**/*_TCI_10m.jp2"))[0]

src = rasterio.open(true_color_path)
src

This returns an open dataset, acting like a pointer to the actual data on disk. Therefore, we are still not loading it.

We can access to several metadata attributes of this dataset, for example:

In [ ]:
src.crs  # Coordinate reference system

In [ ]:
src.bounds  # Expressed in the CRS above

In [ ]:
src.count  # Number of bands

In [ ]:
src.shape  # Shape of the underlying arrays

In [ ]:
src.colorinterp  # Color interpretation of each band

## Vector data

We are interested in downtown Madrid, therefore we will create a rectangle enclosing the Region of Interest (ROI). Using Shapely, we can load our GeoJSON data as raw coordinates:

In [ ]:
from shapely.geometry import shape

roi = shape(roi_geojson)
roi

Next, we have to transform these coordinates into the CRS of the raster data. For that, we need another Python package called pyproj:

In [ ]:
from pyproj import Transformer
from shapely.ops import transform

def reproject_vector(vector, to_epsg, from_epsg=4326):
    # https://shapely.readthedocs.io/en/latest/manual.html#other-transformations
    project = Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(project, roi)

In [ ]:
bounds = reproject_vector(roi, src.crs.to_epsg()).bounds
bounds

We can use these bounds to create a rasterio `Window` object:

In [ ]:
win = src.window(*bounds)
win

And finally, we can use this to load and visualize a partial subset of our data:

In [ ]:
from rasterio.plot import show

show(src.read(window=win), transform=src.transform)

In fact, we can load just one of the bands specifying an integer:

In [ ]:
show(src.read(1, window=win), transform=src.transform, cmap="Reds", title="Red")

Or display them all at the same time:

In [ ]:
import matplotlib.pyplot as plt

fig, (ax_r, ax_g, ax_b) = plt.subplots(ncols=3, figsize=(21, 7))

show(src.read(1, window=win), ax=ax_r, cmap='Reds', title='Red')
show(src.read(2, window=win), ax=ax_g, cmap='Greens', title='Green')
show(src.read(3, window=win), ax=ax_b, cmap='Blues', title='Blue')

## Reading external bands

According to the [Sentinel 2 official definitions](https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/definitions):

> The TCI is an RGB image built from the B02 (Blue), B03 (Green), and B04 (Red) Bands.

Therefore, we could load these bands separately:

In [ ]:
b02 = rasterio.open(list((Path("data") / product_data["filename"]).glob("**/*_B02_10m.jp2"))[0])
b03 = rasterio.open(list((Path("data") / product_data["filename"]).glob("**/*_B03_10m.jp2"))[0])
b04 = rasterio.open(list((Path("data") / product_data["filename"]).glob("**/*_B04_10m.jp2"))[0])

However, the range of these files is much wider:

In [ ]:
src.dtypes

In [ ]:
b04.dtypes

In [ ]:
show(b04.read(window=win), transform=src.transform, cmap='Reds', title='Red')

> The saturation level of 255 digital counts correspond to a level of 3558 for L1C products or 2000 for L2A products (0.3558 and 0.2 in reflectance value respectively.

Therefore, we need to rescale the data manually:

In [ ]:
import numpy as np

In [ ]:
show(
    (b04.read(window=win).clip(max=3558) / 3558 * 255).clip(min=1).astype(np.uint8),
    transform=src.transform,
    cmap='Reds',
    title='Red',
)